# Inference code for YellowKing's model from  DL Sprint 2022
https://www.kaggle.com/code/sameen53/yellowking-dlsprint-inference

In [1]:
!cp -r ../input/python-packages2 ./

In [2]:
!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index
!tar xvfz ./python-packages2/normalizer.tgz
!pip install ./normalizer/bnunicodenormalizer-0.0.24.tar.gz -f ./ --no-index
!tar xvfz ./python-packages2/pyctcdecode.tgz
!pip install ./pyctcdecode/attrs-22.1.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/hypothesis-6.54.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pygtrie-2.5.0.tar.gz -f ./ --no-index --no-deps
!pip install ./pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps

!tar xvfz ./python-packages2/pypikenlm.tgz
!pip install ./pypikenlm/pypi-kenlm-0.1.20220713.tar.gz -f ./ --no-index --no-deps



jiwer/
jiwer/jiwer-2.3.0-py3-none-any.whl
jiwer/python-Levenshtein-0.12.2.tar.gz
jiwer/setuptools-65.3.0-py3-none-any.whl
Looking in links: ./
Processing ./jiwer/jiwer-2.3.0-py3-none-any.whl
normalizer/
normalizer/bnunicodenormalizer-0.0.24.tar.gz
Looking in links: ./
Processing ./normalizer/bnunicodenormalizer-0.0.24.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.0.24-py3-none-any.whl size=17628 sha256=5f0c74ff14825ae376ce0f19d7fcc11c13b79d389cf520e55adfc0425431c591
  Stored in directory: /root/.cache/pip/wheels/3f/7a/fd/fde270138c2a0d8efd0e9ce20f760c2e3a0fc4317975237cb8
Successfully built bnunicodenormalizer
pyctcdecode/
pyctcdecode/hypothesis-6.54.4-py3-none-any.whl
pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl
pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl
pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl
pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
pyctcd

In [3]:
import os
import numpy as np
from tqdm.auto import tqdm
from glob import glob
from transformers import AutoFeatureExtractor, pipeline
import pandas as pd
import librosa
import IPython
from datasets import load_metric
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import gc
import wave
from scipy.io import wavfile
import scipy.signal as sps
import pyctcdecode

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")



In [4]:
# CHANGE ACCORDINGLY
BATCH_SIZE = 16
TEST_DIRECTORY1 = '/kaggle/input/chittagong-1'

path1 = glob(os.path.join(TEST_DIRECTORY1,'*.wav'))
paths = path1
#print(paths[:2])

In [5]:
class CFG:
    my_model_name = '../input/yellowking-dlsprint-model/YellowKing_model'
    processor_name = '../input/yellowking-dlsprint-model/YellowKing_processor'


from transformers import Wav2Vec2ProcessorWithLM

processor = Wav2Vec2ProcessorWithLM.from_pretrained(CFG.processor_name)

In [6]:
my_asrLM = pipeline("automatic-speech-recognition", model=CFG.my_model_name ,feature_extractor =processor.feature_extractor, tokenizer= processor.tokenizer,decoder=processor.decoder ,device=0)


speech, sr = librosa.load('/kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3', sr=processor.feature_extractor.sampling_rate)

my_asrLM([speech]*2, chunk_length_s=112, stride_length_s=None)

my_asrLM



class AudioDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
    def __len__(self):
        return len(self.paths)
    def __getitem__(self,idx):
        speech, sr = librosa.load(self.paths[idx], sr=processor.feature_extractor.sampling_rate) 
#         print(speech.shape)
        return speech


dataset = AudioDataset(paths)
dataset[0]
device = 'cuda:0'

def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ])
    ## padd
    batch = [ torch.Tensor(t) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0)
    return batch, lengths, mask


dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=8, collate_fn=collate_fn_padd)



preds_all = []
for batch, lengths, mask in dataloader:
    preds = my_asrLM(list(batch.numpy().transpose()))
    preds_all+=preds


    from bnunicodenormalizer import Normalizer 


bnorm = Normalizer()
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

def dari(sentence):
    try:
        if sentence[-1]!="।":
            sentence+="।"
    except:
        print(sentence)
    return sentence




df1= pd.DataFrame(
    {
        "External_ID":[p.split(os.sep)[-1].replace('.wav','') for p in paths],
        "content":[p['text']for p in preds_all]
    }
)
df1.content= df1.content.apply(lambda x:normalize(x))
df1.content= df1.content.apply(lambda x:dari(x))

In [7]:
df1

,External_ID,content
0,rec_14_audio_55,তার পেছনে মামাকে গিয়ে আর ফল বানাভালি দুই মাস ম...
1,rec_19_audio_24,ছাত্র নাকি স্কুল স্কুল দিকে ফোটাবেন আর সি আর য...
2,rec_14_audio_90,রেলের পেসারদের নসিবনগর যে ফললেশারআনফরা প্রেজেন...
3,rec_05_audio_0,কিনতে সাকারের বাস পেনিস হতবিলে সাইবার টাইমসে।
4,rec_14_audio_124,আর কী পরিমাণ পরিশ্রম বরণ করে ব্যাক কনফিকার প্র...
...,...,...
995,rec_04_audio_6,ডাম্প মারা পরে অত্যন্ত বারি পতনের কেটে দিবে হা...
996,rec_16_audio_7,শুধু দুই জন এবং বাকি সোনার বাকি সুলিজ হলি মারি...
997,rec_19_audio_108,কেউ জেলার নারীদের চাইতে আরও কিছু পরিমান ইব্রার...
998,rec_14_audio_81,রান সমাকামিতাকে নামে মারবেলের হইতে তুমি কি আর ...


In [8]:
df1.to_csv("submission1.csv", index=False)


In [ ]:
=================================================================================================================================

In [10]:
# CHANGE ACCORDINGLY
BATCH_SIZE = 16
TEST_DIRECTORY2 = '/kaggle/input/chittagong-2'
path2 = glob(os.path.join(TEST_DIRECTORY2,'*.wav'))
paths = path2

In [11]:
my_asrLM = pipeline("automatic-speech-recognition", model=CFG.my_model_name ,feature_extractor =processor.feature_extractor, tokenizer= processor.tokenizer,decoder=processor.decoder ,device=0)


#speech, sr = librosa.load('/kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3', sr=processor.feature_extractor.sampling_rate)

my_asrLM([speech]*2, chunk_length_s=112, stride_length_s=None)

my_asrLM



class AudioDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
    def __len__(self):
        return len(self.paths)
    def __getitem__(self,idx):
        speech, sr = librosa.load(self.paths[idx], sr=processor.feature_extractor.sampling_rate) 
#         print(speech.shape)
        return speech


dataset = AudioDataset(paths)
#dataset[0]
device = 'cuda:0'

def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ])
    ## padd
    batch = [ torch.Tensor(t) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0)
    return batch, lengths, mask


dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=8, collate_fn=collate_fn_padd)



preds_all = []
for batch, lengths, mask in dataloader:
    preds = my_asrLM(list(batch.numpy().transpose()))
    preds_all+=preds


    from bnunicodenormalizer import Normalizer 


bnorm = Normalizer()
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

def dari(sentence):
    try:
        if sentence[-1]!="।":
            sentence+="।"
    except:
        print(sentence)
    return sentence




df2= pd.DataFrame(
    {
        "External_ID":[p.split(os.sep)[-1].replace('.wav','') for p in paths],
        "content":[p['text']for p in preds_all]
    }
)

df2.content= df2.content.apply(lambda x:normalize(x))
df2.content= df2.content.apply(lambda x:dari(x))

In [12]:
df2

,External_ID,content
0,rec_32_audio_29,আর র্যাবের রাবাকে গতরাত্রে যেমত হায়েনারা গানের...
1,rec_26_audio_43,তাই এ জগৎ একটি লাইফ নেয় এক শুনছিলেন।
2,rec_40_audio_29,কিন্তু বালিটিকুরিতে জমানতের সমভাবে মাজুরকাতে ত...
3,rec_27_audio_3,তারা বিভারীকে পারে যে বলেন ফরিদ।
4,rec_26_audio_19,প্রকারে জাতীয় করে তারা বাসা হয় কেলিঅরুবারে কান...
...,...,...
769,rec_30_audio_40,এ নিয়ে সাক্সেনারনাতি কি বা হামব্যাক করে মারিনকে।
770,rec_32_audio_31,রুমে পরবর্তি রাম হবে জানতে সিনারফাক্সের বেশে থ...
771,rec_29_audio_70,কাজের এক প্রকার পরে ইয়েনলি মাবরুর ভযদানি বছরকে...
772,rec_29_audio_10,তা ফ্যাক্টরিতে নারীজাগরগোসসা ঘরের কিলেরকে বাকে...


In [13]:
df2.to_csv("submission2.csv", index=False)


In [14]:
print('done')

done


In [ ]:
============================================================================================================================================

In [15]:
# import pandas as pd
# path1 = "submission1.csv"
# path2 = "submission2.csv"


# df1 = pd.read_csv(path1)
# df2 = pd.read_csv(path2)

values1 = df1[['External_ID','content']]
values2 = df2[['External_ID','content']]


dataframes = [values1, values2]

join = pd.concat(dataframes)
join

,External_ID,content
0,rec_14_audio_55,তার পেছনে মামাকে গিয়ে আর ফল বানাভালি দুই মাস ম...
1,rec_19_audio_24,ছাত্র নাকি স্কুল স্কুল দিকে ফোটাবেন আর সি আর য...
2,rec_14_audio_90,রেলের পেসারদের নসিবনগর যে ফললেশারআনফরা প্রেজেন...
3,rec_05_audio_0,কিনতে সাকারের বাস পেনিস হতবিলে সাইবার টাইমসে।
4,rec_14_audio_124,আর কী পরিমাণ পরিশ্রম বরণ করে ব্যাক কনফিকার প্র...
...,...,...
769,rec_30_audio_40,এ নিয়ে সাক্সেনারনাতি কি বা হামব্যাক করে মারিনকে।
770,rec_32_audio_31,রুমে পরবর্তি রাম হবে জানতে সিনারফাক্সের বেশে থ...
771,rec_29_audio_70,কাজের এক প্রকার পরে ইয়েনলি মাবরুর ভযদানি বছরকে...
772,rec_29_audio_10,তা ফ্যাক্টরিতে নারীজাগরগোসসা ঘরের কিলেরকে বাকে...


In [16]:
join.to_csv("2.4: chittagong-yellowking-inference.csv")

In [17]:
print('done')

done
